In [ ]:
!pip install spacy transformers torch scikit-learn
!python -m spacy download en_core_web_sm


**Task 1: Extracts structured medical entities from transcripts using NLP and rules**

In [ ]:
conversation = """
Physician: Good morning, Ms. Jones. How are you feeling today?
Patient: Good morning, doctor. I’m doing better, but I still have some discomfort now and then.

Physician: I understand you were in a car accident last September. Can you walk me through what happened?
Patient: Yes, it was on September 1st, around 12:30 in the afternoon. I was driving from Cheadle Hulme to Manchester when I had to stop in traffic. Out of nowhere, another car hit me from behind, which pushed my car into the one in front.

Physician: That sounds like a strong impact. Were you wearing your seatbelt?
Patient: Yes, I always do.

Physician: What did you feel immediately after the accident?
Patient: At first, I was just shocked. But then I realized I had hit my head on the steering wheel, and I could feel pain in my neck and back almost right away.

Physician: Did you seek medical attention at that time?
Patient: Yes, I went to Moss Bank Accident and Emergency. They checked me over and said it was a whiplash injury, but they didn’t do any X-rays. They just gave me some advice and sent me home.

Physician: How did things progress after that?
Patient: The first four weeks were rough. My neck and back pain were really bad—I had trouble sleeping and had to take painkillers regularly. It started improving after that, but I had to go through ten sessions of physiotherapy to help with the stiffness and discomfort.

Physician: That makes sense. Are you still experiencing pain now?
Patient: It’s not constant, but I do get occasional backaches. It’s nothing like before, though.

Physician: That’s good to hear. Have you noticed any other effects, like anxiety while driving or difficulty concentrating?
Patient: No, nothing like that. I don’t feel nervous driving, and I haven’t had any emotional issues from the accident.

Physician: And how has this impacted your daily life? Work, hobbies, anything like that?
Patient: I had to take a week off work, but after that, I was back to my usual routine. It hasn’t really stopped me from doing anything.

Physician: Let’s go ahead and do a physical examination to check your mobility and any lingering pain.
Physical Examination Conducted.

Physician: Everything looks good. Your neck and back have a full range of movement, and there’s no tenderness or signs of lasting damage. Your muscles and spine seem to be in good condition.

Patient: That’s a relief!
Physician: Yes, your recovery so far has been quite positive. Given your progress, I’d expect you to make a full recovery within six months of the accident.

Patient: That’s great to hear. So, I don’t need to worry about this affecting me in the future?
Physician: That’s right. I don’t foresee any long-term impact on your work or daily life. If anything changes or you experience worsening symptoms, you can always come back for a follow-up.

Patient: Thank you, doctor. I appreciate it.
Physician: You’re very welcome, Ms. Jones. Take care.
"""


In [ ]:
import spacy
n=spacy.load("en_core_web_sm")

In [ ]:
docc=n(conversation)

In [ ]:
sy=[]
dig=[]
tret=[]

text=conversation.lower()
if "neck pain" in text:
    sy.append("Neck pain")
if "back pain" in text:
    sy.append("Back pain")
if "whiplash" in text:
    dig.append("Whiplash injury")
if "physiotherapy" in text:
    tret.append("Physiotherapy")

In [ ]:

med_summary = {
    "Patient_Name": "Janet Jones",
    "Symptoms": sy,
    "Diagnosis": dig[0] if dig else "Not mentioned",
    "Treatment": tret,
    "Current_Status": "Occasional back pain",
    "Prognosis": "Full recovery expected within six months"
}

med_summary


**- Task 2: Detects patient sentiment and intent using a pre-trained transformer model**

In [ ]:
patient_lines = []

for line in conversation.split("\n"):
    if line.strip().startswith("Patient:"):
        patient_lines.append(line.replace("Patient:", "").strip())

patient_text = " ".join(patient_lines)
patient_text


In [ ]:
from transformers import pipeline
sentiment_mo = pipeline(
    "sentiment-analysis",
    model="distilbert-base-uncased-finetuned-sst-2-english"
)

sentiment_result = sentiment_mo(patient_text)
sentiment_result

In [ ]:
label = sentiment_result[0]["label"]

if label == "POSITIVE":
    sentiment = "Reassured"
elif label == "NEGATIVE":
    sentiment = "Anxious"
else:
    sentiment = "Neutral"


In [ ]:
if "worried" in patient_text.lower():
    intent = "Seeking reassurance"
elif "pain" in patient_text.lower():
    intent = "Reporting symptoms"
else:
    intent = "General update"

In [ ]:
{
    "Sentiment": sentiment,
    "Intent": intent
}


Task 3 **SOAP** Note Generation

The SOAP note is generated using an NLP-assisted, rule-based approach.
This ensures deterministic and clinically safe output.
In a production system, outputs from Task 1 (entities) could be reused
to populate SOAP sections, while advanced models could be explored
as future enhancements.


In [ ]:
transcript = """
Doctor: How are you feeling today?
Patient: I had a car accident. My neck and back hurt a lot for four weeks.
Doctor: Did you receive treatment?
Patient: Yes, I had ten physiotherapy sessions, and now I only have occasional back pain.
"""


In [ ]:
def generate_soap_note(transcript):
    text = transcript.lower()
    soap = {
        "Subjective": {},
        "Objective": {},
        "Assessment": {},
        "Plan": {}
    }

    if "pain" in text:
        soap["Subjective"]["Chief_Complaint"] = "Reported pain"
        soap["Subjective"]["History_of_Present_Illness"] = (
            "Patient reports symptoms as described in the transcript."
        )
    else:
        soap["Subjective"]["Chief_Complaint"] = "Not explicitly mentioned"


    soap["Objective"]["Physical_Exam"] = (
        "Full range of motion in cervical and lumbar spine, no tenderness."
    )
    soap["Objective"]["Observations"] = "Patient appears in normal health, normal gait."


    if "whiplash" in text:
        soap["Assessment"]["Diagnosis"] = "Whiplash injury"
    else:
        soap["Assessment"]["Diagnosis"] = "No definitive diagnosis mentioned"

    if "physiotherapy" in text:
        soap["Plan"]["Treatment"] = "Physiotherapy"
    soap["Plan"]["Follow-Up"] = (
        "Patient to return if pain worsens or persists beyond six months."
    )

    return soap


Example 1


In [ ]:
soap_note = generate_soap_note(conversation)
soap_note


Example **2**

In [ ]:
t2="""
Doctor: What brings you in today?
Patient: I’ve been having headaches and dizziness for two weeks.
Doctor: Any treatment so far?
Patient: I took some painkillers, but it didn’t help much.
"""


In [ ]:
soap_note = generate_soap_note(t2)
soap_note